In [1]:
from datgan import DATGAN
import datgan

import numpy as np
import pandas as pd
import networkx as nx

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('../../data/LPMC/trips.csv', index_col=False)

In [3]:
df.head()

,travel_mode,purpose,fueltype,faretype,bus_scale,travel_year,travel_month,travel_date,day_of_week,start_time_linear,...,dur_driving,cost_transit,cost_driving_fuel,cost_driving_con_charge,driving_traffic_percent,hh_vehicles,hh_borough,hh_income,hh_people,dur_pt_int
0,drive,HBO,Diesel_Car,full,1.0,2012,8,7,2,20.000000,...,0.208611,1.50,0.57,0.0,0.098535,1,Bexley,35-50k,2,0.000000
1,drive,HBW,Diesel_Car,full,1.0,2013,2,8,5,15.000000,...,0.471944,3.00,1.62,0.0,0.354915,1,Harrow,5-10k,3,0.133333
2,pt,HBO,Average_Car,full,0.5,2014,10,8,3,14.000000,...,0.238333,0.75,0.62,0.0,0.212121,0,Lambeth,50-75k,4,0.000000
3,pt,HBE,Average_Car,dis,0.5,2014,3,10,1,10.500000,...,0.308889,0.75,0.60,10.5,0.684353,0,Hackney,10-15k,2,0.000000
4,walk,HBW,Petrol_Car,full,1.0,2013,1,24,4,16.833333,...,0.077500,1.50,0.19,0.0,0.046595,2,Lambeth,50-75k,5,0.000000


In [4]:
# First, define the specificities of continuous variables
data_info = {
    'start_time_linear': {
        'type': 'continuous',
        'bounds': [0.0, 23.999],
        'discrete': False,
    },
    'age': {
        'type': 'continuous',
        'bounds': [0, 100],
        'discrete': True
    },
    'distance': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'discrete': True,
        'apply_func': (lambda x: np.log(x+1))
    },
    'dur_walking': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'dur_cycling': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'dur_pt_access': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'dur_pt_rail': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
    },
    'dur_pt_bus': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
    },
    'dur_pt_int': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
    },
    'dur_driving': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'cost_transit': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
    },
    'cost_driving_fuel': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'enforce_bounds': True,
        'discrete': False,
        'apply_func': (lambda x: np.log(x+1))
    },
    'driving_traffic_percent': {
        'type': 'continuous',
        'bounds': [0, np.infty],
        'discrete': False,
    },
}

# Add the other variables as categorical
for c in df.columns:
    if c not in data_info.keys():
        data_info[c] = {'type': 'categorical'}

In [5]:
# personalised graph
graph = nx.DiGraph()

graph.add_edges_from([
    ("travel_year", "travel_month"),
    ("travel_date", "day_of_week"),
    ("travel_month", "travel_date"),
    ("travel_month", "driving_traffic_percent"),
    ("travel_month", "day_of_week"),
    ("travel_month", "travel_mode"),
    ("travel_date", "day_of_week"),
    ("day_of_week", "driving_traffic_percent"),
    ("day_of_week", "cost_driving_con_charge"),
    ("day_of_week", "purpose"),
    ("day_of_week", "start_time_linear"),
    ("day_of_week", "travel_mode"),
    ("purpose", "distance"),
    ("purpose", "start_time_linear"),
    ("purpose", "travel_mode"),
    ("start_time_linear", "driving_traffic_percent"),
    ("start_time_linear", "cost_driving_con_charge"),
    ("start_time_linear", "travel_mode"),
    ("hh_vehicles", "fueltype"),
    ("hh_vehicles", "driving_license"),
    ("hh_vehicles", "travel_mode"),
    ("fueltype", "cost_driving_con_charge"),
    ("fueltype", "cost_driving_fuel"),
    ("female", "driving_license"),
    ("female", "travel_mode"),
    ("age", "bus_scale"),
    ("age", "driving_license"),
    ("age", "faretype"),
    ("age", "travel_mode"),
    ("driving_license", "travel_mode"),
    ("faretype", "cost_transit"),
    ("faretype", "bus_scale"),
    ("faretype", "travel_mode"),
    ("bus_scale", "cost_transit"),
    ("distance", "cost_driving_fuel"),
    ("distance", "dur_driving"),
    ("distance", "dur_walking"),
    ("distance", "dur_cycling"),
    ("distance", "dur_pt_access"),
    ("distance", "dur_pt_rail"),
    ("distance", "dur_pt_bus"),
    ("distance", "dur_pt_int"),
    ("distance", "pt_n_interchanges"),
    ("distance", "travel_mode"),
    ("pt_n_interchanges", "dur_pt_rail"),
    ("pt_n_interchanges", "dur_pt_bus"),
    ("pt_n_interchanges", "dur_pt_int"),
    ("pt_n_interchanges", "cost_transit"),
    ("driving_traffic_percent", "cost_driving_con_charge"),
    ("driving_traffic_percent", "travel_mode"),
    ("cost_driving_fuel", "cost_driving_con_charge"),
    ("cost_driving_fuel", "travel_mode"),
    ("cost_driving_con_charge", "travel_mode"),
    ("dur_driving", "travel_mode"),
    ("dur_walking", "travel_mode"),
    ("dur_cycling", "travel_mode"),
    ("dur_pt_access", "travel_mode"),
    ("dur_pt_rail", "cost_transit"),
    ("dur_pt_rail", "travel_mode"),
    ("dur_pt_bus", "cost_transit"),
    ("dur_pt_bus", "travel_mode"),
    ("dur_pt_int", "travel_mode"),
    ("cost_transit", "travel_mode"),
    ("hh_borough", "hh_income"),
    ("hh_borough", "travel_mode"),
    ("hh_borough", "distance"),
    ("hh_borough", "hh_people"),
    ("hh_income", "hh_vehicles"),
    ("hh_income", "age"),
    ("hh_income", "hh_people"),
    ("hh_people", "age"),
    ("hh_people", "female"),
    ("hh_people", "hh_vehicles")
])

In [6]:
name = 'LPMC_cond'

In [7]:
output_folder = '../output/{}/'.format(name)

In [8]:
datgan = DATGAN(output=output_folder,
                loss_function='WGGP',
                conditional_inputs=['age', 'female', 'hh_borough'],
                batch_size=1101,
                num_epochs=1000)

In [9]:
datgan.fit(df, data_info, graph, preprocessed_data_path='../output/encoded_LPMC')

Preprocessed data have been loaded!
Start training DATGAN with the WGGP loss (05/05/2022 13:17:25).
Restored models from epoch 1000.


Training DATGAN: 0it [00:00, ?it/s]

DATGAN has finished training (05/05/2022 13:17:28) - Training time: 03 seconds


In [11]:
for i in range(5):
    samp = datgan.sample(len(df), inputs=df[datgan.conditional_inputs], randomize=False)
    samp.to_csv('../../data/synthetic/test/ciDATGAN_{:02d}.csv'.format(i+1), index=False)


Sampling from DATGAN: 100%|██████████| 16904/16904 [00:22<00:00, 760.04it/s]

Sampling from DATGAN: 100%|██████████| 16904/16904 [00:21<00:00, 770.87it/s]

Sampling from DATGAN: 100%|██████████| 16904/16904 [00:21<00:00, 778.59it/s]

Sampling from DATGAN: 100%|██████████| 16904/16904 [00:21<00:00, 777.16it/s]

Sampling from DATGAN: 100%|██████████| 16904/16904 [00:24<00:00, 687.26it/s]


## All the boroughs

In [10]:
count = 1
for r in df.hh_borough.unique():

    tmp = pd.read_csv('../../data/nomis/{}.csv'.format(r))

    print('Sampling for {} ({}/{})'.format(r, count, len(df.hh_borough.unique())))

    samp = datgan.sample(len(tmp), inputs=tmp, randomize=False)

    samp.to_csv('../../data/synthetic/ciDATGAN/{}.csv'.format(r), index=False)

    count += 1

Sampling for Bexley (1/33)


Sampling from DATGAN: 100%|██████████| 231997/231997 [04:44<00:00, 815.27it/s]


Sampling for Harrow (2/33)


Sampling from DATGAN: 100%|██████████| 239056/239056 [04:47<00:00, 830.07it/s]


Sampling for Lambeth (3/33)


...

Sampling from DATGAN: 100%|██████████| 7375/7375 [00:11<00:00, 635.93it/s]


Sampling for Lewisham (32/33)


Sampling from DATGAN: 100%|██████████| 275885/275885 [05:31<00:00, 833.24it/s]


Sampling for Westminster (33/33)


Sampling from DATGAN: 100%|██████████| 219396/219396 [04:21<00:00, 839.07it/s]
